In [1]:
import casadi as ca
import SimpleOpt as so
import numpy as np
import sys
import matplotlib.pyplot as plt
sys.path.append('/Users/jeremy/git/python-control/')
import control as ctrl

def trapInt(t,inVec):
  sumval = 0.
  for ii in range(0,inVec.shape[1]-1):
      sumval = sumval + (inVec[ii]+inVec[ii+1])/2.0 * (t[ii+1]-t[ii])
  return sumval
  
# Constants
p_0 = 0    # Initial position
v_0 = 0    # Initial velocity
p_f = 1    # Final position
v_f = 0    # Final velocity (at step nm)

# Eye movements
n = 100 # total of 100 ms
nm = 99 # finish at index nm
dt = 1/n # in s
Bs  = 0
M   = 1
eyefrresults = []
# Create JuMP model, using Ipopt as the solver
#eyefr = Model(with_optimizer(Ipopt.Optimizer, print_level=0))

opti = ca.Opti()
p = opti.variable(n,1)
v = opti.variable(n,1) 
F = opti.variable(n,1) 
Fdot = opti.variable(n,1) 
Fddot = opti.variable(n-1,1) 
V11 = opti.variable(n-1,1) 
Ek = opti.variable(n-1,1) 
slackFddp = opti.variable(n-1,1) 
slackFddn = opti.variable(n-1,1) 
posPowJ = opti.variable(n-1,1) 
slackPosPowJ = opti.variable(n-1,1) 
T = 1

  # Start with continuous-time state space, then discrete time
A = np.array([[0, 1, 0, 0], [0, -Bs/M, 1/M, 0], [0, 0, 0, 1], [0, 0, 0, 0]])

# position, velocity, F, Fdot just with damping
B = np.array([[0], [0], [0], [1]])
C = np.array([1,0, 0, 0])
D = [0]
#sseye = ctrl.ss(A, B, C, D)
#ssdeye = ctrl.c2d(sseye, Δt)[1,1] # exp(A Δt)



In [2]:
C = np.array([1,0, 0, 0])
sseye = ctrl.ss(A,B,C,D)
ssdeye = ctrl.c2d(sseye,dt)
ssdeye.A

dimMat      = ca.MX.zeros(4,4)
dimMat[1,1] = T
dimMat[2,2] = T
dimMat[3,3] = T

def matpow(inmat,n):
  inmatp = inmat
  for j in range(0,n-1):
    inmatp=inmatp@inmat
  return inmatp

V11expr = ca.MX.zeros(n-1,n-1)

# converting loops to python. 
# first question: is the iterate used primarily for math, or for indexing?
# sometimes it's both. 
# if indexing, then convert the for-loop from matlab[m,n] to python[m-1,n). 
# if math, 
  # then convert the looprange from matlab[m,n] to python[m,n+1]
  # and manually adjust any indices by -1.
# these loops are for math. 
for myt in range(2,n+1): #convert the loop for math
  for i in range(1,myt-1): #convert the loop for math
    AB = matpow(ssdeye.A @ dimMat,myt-i-1) @ (dimMat @ ssdeye.B)
    ABBA = AB @ AB.T
    V11expr[myt-1-1,i-1] = ABBA[0,0]*F[i-1]*F[i-1] #manually adjust the indices

In [3]:

# 1. Initial conditions and final conditions
opti.subject_to(p[0]  == 0 )
opti.subject_to(v[0]  == 0 )
opti.subject_to(p[nm] == p_f)
opti.subject_to(v[nm]  == 0 )
opti.subject_to(F[0]  == 0 )
opti.subject_to(Fdot[0]  == 0 )
opti.subject_to(F[nm]  == 0 )
opti.subject_to(Fdot[nm]  == 0 )



In [4]:

# 2. hold period
for j in np.arange(nm,n): # julia nm+1:n # during hold period, keep velocity zero
    opti.subject_to(v[j] == 0)



In [5]:

# 3. slack vars for fdotdot
for j in np.arange(0,n-1): # during hold period, keep velocity zero
  opti.subject_to(slackFddp[j] >=Fddot[j])
  opti.subject_to(slackFddp[j] >=0)
  opti.subject_to(slackFddn[j] <=Fddot[j])
  opti.subject_to(slackFddn[j] <=0)



In [6]:
# 4. Dynamics, as state-space linear system
for j in np.arange(1,n): #julia 2:n
    opti.subject_to(p[j] == p[j-1] + T*dt*v[j-1])
    opti.subject_to(v[j] == v[j-1] + T*dt*(-Bs*v[j-1] + F[j-1])) #EOM
    opti.subject_to(F[j] == F[j-1] + T*dt*Fdot[j-1])
    opti.subject_to(Fdot[j] == Fdot[j-1] + T*dt*Fddot[j-1])

In [7]:
#6. all expressions for variance, energy need to be set here. 
for j in np.arange(0,n-1):#julia j in 1:n-1
  opti.subject_to(V11[j] == ca.sum2(V11expr[j,1:j]))   # setting covariance11
  opti.subject_to(posPowJ[j]  == v[j] * F[j] * T)
  opti.subject_to(slackPosPowJ[j] >=posPowJ[j])
  opti.subject_to(slackPosPowJ[j] >=0)



In [ ]:
#7. Objectives
tvec    = np.linspace(0,1,num=100)*T

kFR     = 1e-1 #bells for 1e-1, together with T = 1, work = 8
objFrJ  = kFR * trapInt(tvec[0:-1], -slackFddn + slackFddp) * T

kPosWorkW   = 4.2
objPosWorkJ = kPosWorkW * trapInt(tvec[1:-1],slackPosPowJ)

kT       = 1
objTimeJ = kT*T

varianceHold = ca.sum1(V11[nm:-1])

#minimize Fdot squared over movement period
opti.minimize(objFrJ + objPosWorkJ + objTimeJ)
# opti.minimize(ca.sumsqr(Fddot))
#opti.subject_to()



In [ ]:
maxIter = 1000
pOpt = {"expand":True}
sOpt = {"max_iter"        : maxIter,
        "constr_viol_tol" : 1e-2,
        "dual_inf_tol"    : 1e-2}
opti.solver('ipopt',pOpt,sOpt)
f,ax = plt.subplots(2,2)
def callbackPlots(i):
    ax[0,0].plot(tvec,opti.debug.value(p))
    ax[0,1].plot(tvec,opti.debug.value(v),color=(1,.8-.8*i/(maxIter),1))
    ax[1,0].plot(tvec[0:-1],opti.debug.value(V11),color=(1,.8-.8*i/(maxIter),1))
    ax[1,1].plot(tvec[0:-1],opti.debug.value(Fddot),color=(1,.8-.8*i/(maxIter),1))
opti.callback(callbackPlots)

try:
      sol = opti.solve()
except:
  print("did not find solution")

# p_opt = opti.value(p)
# fig, ax = plt.subplots()
# p1 = plt.plot(tvec, p)
# p2 = plot(tvec, [res.v for res in eyefrresults], xlabel="t", ylabel="velocity")
# p3 = plot(tvec, [res.F for res in eyefrresults], xlabel="t", ylabel="F")
# p4 = plot(tvec[1:n-1], [res.Fddot[1:n-1] for res in eyefrresults], xlabel="t", ylabel="Fddot")
# plot(p1,p2,p3,p4,layout=(2,2),label="")



In [ ]:
max(opti.value(V11))
